In [ ]:
import pandas as pd

# Load cleaned dataset
balls = pd.read_csv("clean_balls.csv")

# Extract unique teams
teams = sorted(balls["batting_team"].dropna().unique())

print("\nAvailable Teams:")
for i, team in enumerate(teams, start=1):
    print(f"{i}. {team}")

choice_a = int(input("\nSelect Team A (Enter number): "))
choice_b = int(input("Select Team B (Enter number): "))

TEAM_A = teams[choice_a - 1]
TEAM_B = teams[choice_b - 1]

print(f"\n✅ You selected: {TEAM_A} vs {TEAM_B}")

# Function to calculate role-based Best XI
def get_best_xi(team_name, balls_df):
    batting = balls_df[balls_df["batting_team"] == team_name]
    batting_scores = batting.groupby("batsman").agg(runs=("batsman_runs", "sum")).reset_index()

    bowling = balls_df[balls_df["bowling_team"] == team_name]
    bowling_scores = bowling[bowling["is_wicket"] == 1].groupby("bowler").size().reset_index(name="wickets")

    data = pd.merge(batting_scores, bowling_scores, left_on="batsman", right_on="bowler", how="outer").fillna(0)
    data["player"] = data["batsman"]
    data["runs"] = data["runs"]
    data["wickets"] = data["wickets"]
    
    # Role classification
    def assign_role(row):
        if row["wickets"] >= 50 and row["runs"] < 500:
            return "Bowler"
        elif row["runs"] >= 2000 and row["wickets"] < 20:
            return "Batsman"
        elif row["runs"] >= 1000 and row["wickets"] >= 30:
            return "All-Rounder"
        else:
            return "Batsman" if row["runs"] > row["wickets"] * 20 else "Bowler"

    data["role"] = data.apply(assign_role, axis=1)

    # Picking XI with constraints
    bowlers = data[data["role"] == "Bowler"].nlargest(5, "wickets")
    batsmen = data[data["role"] == "Batsman"].nlargest(4, "runs")
    all_rounders = data[data["role"] == "All-Rounder"].nlargest(2, ["runs", "wickets"])

    xi = pd.concat([bowlers, batsmen, all_rounders]).head(11)
    return xi[["player", "role", "runs", "wickets"]]

# Generate Best XI for both teams
best_xi_a = get_best_xi(TEAM_A, balls)
best_xi_b = get_best_xi(TEAM_B, balls)

print(f"\n🏏 Best XI - {TEAM_A}")
print(best_xi_a)

print(f"\n🏏 Best XI - {TEAM_B}")
print(best_xi_b)



Available Teams:
1. Chennai Super Kings
2. Deccan Chargers
3. Delhi Capitals
4. Gujarat Lions
5. Gujarat Titans
6. Kings XI Punjab
7. Kochi Tuskers Kerala
8. Kolkata Knight Riders
9. Lucknow Super Giants
10. Mumbai Indians
11. Pune Warriors
12. Rajasthan Royals
13. Rising Pune Supergiants
14. Royal Challengers Bangalore
15. Sunrisers Hyderabad


/var/folders/3l/67y0958d03l9598gqq_b3f6m0000gn/T/ipykernel_25381/1429125008.py:4: DtypeWarning: Columns (12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  balls = pd.read_csv("clean_balls.csv")
